In [1]:
import pyspark
from pyspark.sql import SparkSession
import warnings
from pyspark.sql.types import StructField, StructType, IntegerType, DateType, StringType, TimestampType, DoubleType, LongType
import pyspark.sql.functions as F
warnings.filterwarnings("ignore")

In [2]:
spark = SparkSession.builder.master("local[*]")\
                    .appName("hw").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/04 20:40:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
## Q1
spark.version

'3.3.2'

In [ ]:
schema = StructType(
    [StructField('dispatching_base_num', StringType(), True),
     StructField('pickup_datetime', TimestampType(), True),
     StructField('dropoff_datetime', TimestampType(), True),
     StructField('PULocationID', IntegerType(), True),
     StructField('DOLocationID', IntegerType(), True),
     StructField('SR_Flag', StringType(), True),
     StructField('Affiliated_base_number', StringType(), True)]
)

In [8]:
df = spark.read.option("header", "true").schema(schema).csv("/home/Ezz/week_5/data/fhvhv_tripdata_2021-06.csv")
df.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02764|2021-06-01 00:02:41|2021-06-01 00:07:46|         174|          18|      N|                B02764|
|              B02764|2021-06-01 00:16:16|2021-06-01 00:21:14|          32|         254|      N|                B02764|
|              B02764|2021-06-01 00:27:01|2021-06-01 00:42:11|         240|         127|      N|                B02764|
|              B02764|2021-06-01 00:46:08|2021-06-01 00:53:45|         127|         235|      N|                B02764|
|              B02510|2021-06-01 00:45:42|2021-06-01 01:03:33|         144|         146|      N|                  null|
+--------------------+------------------

In [10]:
## Q2
df.repartition(12).write.parquet("/home/Ezz/week_5/data/fhvhv_tripdata_202106_partitioned")

In [11]:
!ls -lh /home/Ezz/week_5/data/fhvhv_tripdata_202106_partitioned

total 271M
-rw-r--r-- 1 Ezz Ezz   0 Mar  4 20:49 _SUCCESS
-rw-r--r-- 1 Ezz Ezz 23M Mar  4 20:49 part-00000-2cf5d23d-0848-4657-86a2-94814c46f7c1-c000.snappy.parquet
-rw-r--r-- 1 Ezz Ezz 23M Mar  4 20:49 part-00001-2cf5d23d-0848-4657-86a2-94814c46f7c1-c000.snappy.parquet
-rw-r--r-- 1 Ezz Ezz 23M Mar  4 20:49 part-00002-2cf5d23d-0848-4657-86a2-94814c46f7c1-c000.snappy.parquet
-rw-r--r-- 1 Ezz Ezz 23M Mar  4 20:49 part-00003-2cf5d23d-0848-4657-86a2-94814c46f7c1-c000.snappy.parquet
-rw-r--r-- 1 Ezz Ezz 23M Mar  4 20:49 part-00004-2cf5d23d-0848-4657-86a2-94814c46f7c1-c000.snappy.parquet
-rw-r--r-- 1 Ezz Ezz 23M Mar  4 20:49 part-00005-2cf5d23d-0848-4657-86a2-94814c46f7c1-c000.snappy.parquet
-rw-r--r-- 1 Ezz Ezz 23M Mar  4 20:49 part-00006-2cf5d23d-0848-4657-86a2-94814c46f7c1-c000.snappy.parquet
-rw-r--r-- 1 Ezz Ezz 23M Mar  4 20:49 part-00007-2cf5d23d-0848-4657-86a2-94814c46f7c1-c000.snappy.parquet
-rw-r--r-- 1 Ezz Ezz 23M Mar  4 20:49 part-00008-2cf5d23d-0848-4657-86a2-94814c46f7c1-c000.sna

In [17]:
## Q3
df = spark.read.option("header", "true").parquet("/home/Ezz/week_5/data/fhvhv_tripdata_202106_partitioned")
df = df.withColumn("pickup_date", F.to_date(df["pickup_datetime"]))\
       .withColumn("dropoff_date", F.to_date(df["dropoff_datetime"]))
df.filter(df["pickup_date"] == F.lit("2021-06-15")).count()

452470

In [21]:
df.createOrReplaceTempView("data")

In [23]:
df = df.withColumn("DiffInSeconds", df["dropoff_datetime"].cast("long") - df["pickup_datetime"].cast("long"))
df.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+------------+-------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|pickup_date|dropoff_date|DiffInSeconds|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+------------+-------------+
|              B02617|2021-06-04 16:50:34|2021-06-04 17:01:18|         118|         109|      N|                B02617| 2021-06-04|  2021-06-04|          644|
|              B02875|2021-06-02 22:28:45|2021-06-02 22:37:28|         163|          79|      N|                B02875| 2021-06-02|  2021-06-02|          523|
|              B02871|2021-06-03 11:47:48|2021-06-03 11:52:23|         231|          13|      N|                B02871| 2021-06-03|  2021-06-03|          275|
|              B02888|2021-06-03 08:45:25|2021

In [26]:
df = df.withColumn("DiffInHours", df["diffInSeconds"]/3600)

In [27]:
## Q4
df.select("diffInHours").orderBy(F.col("diffInHours").desc()).show(1)

+----------------+
|     diffInHours|
+----------------+
|66.8788888888889|
+----------------+
only showing top 1 row



In [28]:
## Q5
## 4040

In [29]:
location_lookup = spark.read.option("header", "true").csv("/home/Ezz/week_5/data/taxi+_zone_lookup.csv")
location_lookup.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [30]:
location_lookup.createOrReplaceTempView("lookup")

In [32]:
## Q6
spark.sql("""
SELECT temp.Zone,COUNT(base.PULocationID)
FROM data AS base
LEFT JOIN lookup AS temp ON temp.LocationID = base.PULocationID
GROUP BY 1
ORDER BY 2 DESC
""").show(1)

+-------------------+-------------------+
|               Zone|count(PULocationID)|
+-------------------+-------------------+
|Crown Heights North|             231279|
+-------------------+-------------------+
only showing top 1 row

